In [20]:
import camb
from camb import model, initialpower
import pysm3
import pysm3.units as u
import healpy as hp
import numpy as np
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import math

np.set_printoptions(16)

Nside=512
ls=np.arange(1,3*Nside+1)

In [21]:
cp1=camb.read_ini('planck_2018_acc_np_params.ini')  

#compare with primordial B mode
inflation_params = initialpower.InitialPowerLaw()
inflation_params.set_params( r=0.1)
results = camb.get_transfer_functions(cp1)
results.power_spectra_from_transfer(inflation_params)

clstand= results.get_unlensed_total_cls(1535,CMB_unit='muK')
clbb=clstand[:,2]/ls/(ls+1)*2*np.pi

# Restore Cls
### Run other notebooks first, then restore the data from them.

In [34]:
%store -r noiseCL    
#based on LiteBIRD 2 muK arcmin, can transfer to sensitivity x by (x/2)^2

%store -r foreCL
foreCL=foreCL[0:1536]   #cutoff

%store -r lenCL   
lenCL=lenCL[0:1536]
#lenCL is with l(l+1)/2pi
lenCL=lenCL/ls/(ls+1)*2*np.pi


#polarization angle
%store -r unlensedTQU_TEB
%store -r unlensedTQU_alpha_TEB     
#LiteBIRD
%store -r unlensedTQU_DW_alpha_TEB   
#DW, LAT, same resolution
%store -r unlensedTQU_UD_alpha_TEB   
#UD
%store -r unlensedTQU_SAT_alpha_TEB  
#SAT
%store -r unlensedTQU_PICO_alpha_TEB 
#PICO 


#pointing error
%store -r TEB_pointing
%store -r TEB_SAT_pointing
%store -r TEB_PICO_pointing


%store -r noises_map_140GHz_10FSL_mmasked_B_filtered
%store -r noises_map_140GHz_20FSL_mmasked_B_filtered
%store -r noises_map_140GHz_15FSL_mmasked_B_filtered
%store -r noises_map_100GHz_10FSL_mmasked_B_filtered
%store -r noises_map_100GHz_20FSL_mmasked_B_filtered
%store -r noises_map_100GHz_15FSL_mmasked_B_filtered
%store -r noises_map_195GHz_10FSL_mmasked_B_filtered
%store -r noises_map_195GHz_20FSL_mmasked_B_filtered
%store -r noises_map_195GHz_15FSL_mmasked_B_filtered

%store -r noises_map_225GHz_10FSL_mmasked_TEB_DW
%store -r noises_map_225GHz_20FSL_mmasked_TEB_DW
%store -r noises_map_225GHz_15FSL_mmasked_TEB_DW
%store -r noises_map_220GHz_10FSL_mmasked_TEB_DW
%store -r noises_map_220GHz_20FSL_mmasked_TEB_DW
%store -r noises_map_220GHz_15FSL_mmasked_TEB_DW
%store -r noises_map_145GHz_10FSL_mmasked_TEB_DW
%store -r noises_map_145GHz_20FSL_mmasked_TEB_DW
%store -r noises_map_145GHz_15FSL_mmasked_TEB_DW
%store -r noises_map_95GHz_10FSL_mmasked_TEB_DW
%store -r noises_map_95GHz_20FSL_mmasked_TEB_DW
%store -r noises_map_95GHz_15FSL_mmasked_TEB_DW
%store -r noises_map_93GHz_10FSL_mmasked_TEB_DW
%store -r noises_map_93GHz_20FSL_mmasked_TEB_DW
%store -r noises_map_93GHz_15FSL_mmasked_TEB_DW

%store -r noises_map_225GHz_10FSL_mmasked_TEB_SAT
%store -r noises_map_225GHz_20FSL_mmasked_TEB_SAT
%store -r noises_map_225GHz_15FSL_mmasked_TEB_SAT
%store -r noises_map_145GHz_10FSL_mmasked_TEB_SAT
%store -r noises_map_145GHz_20FSL_mmasked_TEB_SAT
%store -r noises_map_145GHz_15FSL_mmasked_TEB_SAT
%store -r noises_map_93GHz_10FSL_mmasked_TEB_SAT
%store -r noises_map_93GHz_20FSL_mmasked_TEB_SAT
%store -r noises_map_93GHz_15FSL_mmasked_TEB_SAT

%store -r noises_map_220GHz_10FSL_mmasked_TEB_UD
%store -r noises_map_220GHz_20FSL_mmasked_TEB_UD
%store -r noises_map_220GHz_15FSL_mmasked_TEB_UD
%store -r noises_map_145GHz_10FSL_mmasked_TEB_UD
%store -r noises_map_145GHz_20FSL_mmasked_TEB_UD
%store -r noises_map_145GHz_15FSL_mmasked_TEB_UD
%store -r noises_map_95GHz_10FSL_mmasked_TEB_UD
%store -r noises_map_95GHz_20FSL_mmasked_TEB_UD
%store -r noises_map_95GHz_15FSL_mmasked_TEB_UD

%store -r noises_map_223GHz_10FSL_mmasked_TEB_PICO
%store -r noises_map_223GHz_20FSL_mmasked_TEB_PICO
%store -r noises_map_223GHz_15FSL_mmasked_TEB_PICO
%store -r noises_map_155GHz_10FSL_mmasked_TEB_PICO
%store -r noises_map_155GHz_20FSL_mmasked_TEB_PICO
%store -r noises_map_155GHz_15FSL_mmasked_TEB_PICO
%store -r noises_map_90GHz_10FSL_mmasked_TEB_PICO
%store -r noises_map_90GHz_20FSL_mmasked_TEB_PICO
%store -r noises_map_90GHz_15FSL_mmasked_TEB_PICO

In [23]:
FWHM=np.array([30,1.4,1.5,1.4,17,6.2])/60/180*np.pi
sig=FWHM/(2*np.sqrt(2*np.log(2)))

def ant(x):
    return np.exp(-x*(x+1)*sig**2)

# Foreground and sensitivity, camparison baseline

In [24]:
# foreground lensing
ss1=0
for i in ls: 
    s1=(2*i+1)/2*(clbb[i-1]/(foreCL[i-1]+lenCL[i-1]))**2
    ss1=ss1+s1
0.1*ss1**(-1/2)

0.0005709401516069272

In [46]:
# foreground lensing sensitivity     no component separation nor delensing

ss1_lite0=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*foreCL[i-1]+ant(i-1)[0]*lenCL[i-1]+noiseCL))**2
    ss1_lite0=ss1_lite0+s1
print(0.1*ss1_lite0**(-1/2)*1/np.sqrt(0.6))    #0.6 sky fraction

#2.8 uK arcmin DW
ss1_s4dw0=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*foreCL[i-1]+ant(i-1)[1]*lenCL[i-1]+(2.8/2)**2*noiseCL))**2
    ss1_s4dw0=ss1_s4dw0+s1
print(0.1*ss1_s4dw0**(-1/2)*1/np.sqrt(0.6))

#0.96 uK arcmin  UD
ss1_s4ud0=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*foreCL[i-1]+ant(i-1)[2]*lenCL[i-1]+(0.96/2)**2*noiseCL))**2
    ss1_s4ud0=ss1_s4ud0+s1
print(0.1*ss1_s4ud0**(-1/2)*1/np.sqrt(0.03))



#6 uK arcmin  LAT
ss1_solat0=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*foreCL[i-1]+ant(i-1)[3]*lenCL[i-1]+(6/2)**2*noiseCL))**2
    ss1_solat0=ss1_solat0+s1
print(0.1*ss1_solat0**(-1/2)*1/np.sqrt(0.4))

#2 uK arcmin  SAT
ss1_sosat0=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*foreCL[i-1]+ant(i-1)[4]*lenCL[i-1]+(2/2)**2*noiseCL))**2
    ss1_sosat0=ss1_sosat0+s1
print(0.1*ss1_sosat0**(-1/2)*1/np.sqrt(0.1))


#PICO
ss1_pico0=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*foreCL[i-1]+ant(i-1)[5]*lenCL[i-1]+(0.61/2)**2*noiseCL))**2
    ss1_pico0=ss1_pico0+s1
print(0.1*ss1_pico0**(-1/2)*1/np.sqrt(0.6))

0.0008329594071121042
0.0009049738128949849
0.003385886391020811
0.0018120150725846264
0.002024099743664277
0.0007452131155539646


In [47]:
# foreground lensing sensitivity      component separation
RF=np.array([0.01,0.08,0.01,0.43,0.05,0.001])


ss1_lite1=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*lenCL[i-1]+noiseCL))**2
    ss1_lite1=ss1_lite1+s1
print(0.1*ss1_lite1**(-1/2)*1/np.sqrt(0.6))

#2.8 uK arcmin DW
ss1_s4dw1=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*lenCL[i-1]+(2.8/2)**2*noiseCL))**2
    ss1_s4dw1=ss1_s4dw1+s1
print(0.1*ss1_s4dw1**(-1/2)*1/np.sqrt(0.6))

#0.96 uK arcmin  UD
ss1_s4ud1=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*lenCL[i-1]+(0.96/2)**2*noiseCL))**2
    ss1_s4ud1=ss1_s4ud1+s1
print(0.1*ss1_s4ud1**(-1/2)*1/np.sqrt(0.03))



#6 uK arcmin  LAT
ss1_solat1=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*lenCL[i-1]+(6/2)**2*noiseCL))**2
    ss1_solat1=ss1_solat1+s1
print(0.1*ss1_solat1**(-1/2)*1/np.sqrt(0.4))

#2 uK arcmin  SAT
ss1_sosat1=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*lenCL[i-1]+(2/2)**2*noiseCL))**2
    ss1_sosat1=ss1_sosat1+s1
print(0.1*ss1_sosat1**(-1/2)*1/np.sqrt(0.1))


#PICO
ss1_pico1=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*lenCL[i-1]+(0.61/2)**2*noiseCL))**2
    ss1_pico1=ss1_pico1+s1
print(0.1*ss1_pico1**(-1/2)*1/np.sqrt(0.6))

0.000420573201301006
0.0005929221365175014
0.0016842768097305788
0.001567499626009523
0.001235167900748779
0.00017169502940465986


In [48]:
# foreground lensing sensitivity   + normal delensing
RF=np.array([0.01,0.08,0.01,0.43,0.05,0.001])
Dauto=np.array([0.73,0.38,0.12,1,0.59,0.20])

ss1_lite=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL))**2
    ss1_lite=ss1_lite+s1
print(0.1*ss1_lite**(-1/2)*1/np.sqrt(0.6))

#2.8 uK arcmin DW
ss1_s4dw=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL))**2
    ss1_s4dw=ss1_s4dw+s1
print(0.1*ss1_s4dw**(-1/2)*1/np.sqrt(0.6))

#0.96 uK arcmin  UD
ss1_s4ud=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL))**2
    ss1_s4ud=ss1_s4ud+s1
print(0.1*ss1_s4ud**(-1/2)*1/np.sqrt(0.03))



#6 uK arcmin  LAT
ss1_solat=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL))**2
    ss1_solat=ss1_solat+s1
print(0.1*ss1_solat**(-1/2)*1/np.sqrt(0.4))

#2 uK arcmin  SAT
ss1_sosat=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL))**2
    ss1_sosat=ss1_sosat+s1
print(0.1*ss1_sosat**(-1/2)*1/np.sqrt(0.1))

#PICO
ss1_pico=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL))**2
    ss1_pico=ss1_pico+s1
print(0.1*ss1_pico**(-1/2)*1/np.sqrt(0.6))

0.00034021956526983214
0.000340775131451545
0.0003178507655555055
0.001567499626009523
0.0008376933018661762
6.609071658101006e-05


In [49]:
# foreground lensing sensitivity   + minimum delensing
RF=np.array([0.01,0.08,0.01,0.43,0.05,0.001])
Dmin=np.array([0.33,0.24,0.04,1,0.22,0.03])

ss1_lite2=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dmin[0]*lenCL[i-1]+noiseCL))**2
    ss1_lite2=ss1_lite2+s1
print(0.1*ss1_lite2**(-1/2)*1/np.sqrt(0.6))

#2.8 uK arcmin DW
ss1_s4dw2=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dmin[1]*lenCL[i-1]+(2.8/2)**2*noiseCL))**2
    ss1_s4dw2=ss1_s4dw2+s1
print(0.1*ss1_s4dw2**(-1/2)*1/np.sqrt(0.6))

#0.96 uK arcmin  UD
ss1_s4ud2=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[0]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dmin[2]*lenCL[i-1]+(0.96/2)**2*noiseCL))**2
    ss1_s4ud2=ss1_s4ud2+s1
print(0.1*ss1_s4ud2**(-1/2)*1/np.sqrt(0.03))



#6 uK arcmin  LAT
ss1_solat2=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dmin[3]*lenCL[i-1]+(6/2)**2*noiseCL))**2
    ss1_solat2=ss1_solat2+s1
print(0.1*ss1_solat2**(-1/2)*1/np.sqrt(0.4))

#2 uK arcmin  SAT
ss1_sosat2=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dmin[4]*lenCL[i-1]+(2/2)**2*noiseCL))**2
    ss1_sosat2=ss1_sosat2+s1
print(0.1*ss1_sosat2**(-1/2)*1/np.sqrt(0.1))


#PICO
ss1_pico2=0
for i in ls: 
    s1=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dmin[5]*lenCL[i-1]+(0.61/2)**2*noiseCL))**2
    ss1_pico2=ss1_pico2+s1
print(0.1*ss1_pico2**(-1/2)*1/np.sqrt(0.6))

0.00020667140081942977
0.0002822886757910645
0.00017111099372695235
0.001567499626009523
0.0004673849984659818
1.8772290520076938e-05


# Polarization angle

In [50]:
# polarization angle caused r uncertainty percentage
# [1,3,5,7,10,12,14,16,18,20,22,24,26,28,30,60,90] deg

#LiteBIRD
ss_pol_lite=np.zeros(17)
for j in np.arange(len(ss_pol_lite)):
    for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+unlensedTQU_alpha_TEB[j][2,i-1]))**2
        ss_pol_lite[j]=ss_pol_lite[j]+s4
print('LiteBIRD:', (ss_pol_lite/ss1_lite)**(-1/2)-1)

#DW
ss_pol_DW=np.zeros(17)
for j in np.arange(len(ss_pol_DW)):
    for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+unlensedTQU_DW_alpha_TEB[j][2,i-1]))**2
        ss_pol_DW[j]=ss_pol_DW[j]+s4
print('DW:', (ss_pol_DW/ss1_s4dw)**(-1/2)-1)

#UD
ss_pol_UD=np.zeros(17)
for j in np.arange(len(ss_pol_UD)):
    for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+unlensedTQU_UD_alpha_TEB[j][2,i-1]))**2
        ss_pol_UD[j]=ss_pol_UD[j]+s4
print('UD:', (ss_pol_UD/ss1_s4ud)**(-1/2)-1)


#LAT (POLAR NOISE SAME AS DW BUT SENSITIVITY IS DIFFERENT)
ss_pol_LAT=np.zeros(17)
for j in np.arange(len(ss_pol_LAT)):
    for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+unlensedTQU_DW_alpha_TEB[j][2,i-1]))**2
        ss_pol_LAT[j]=ss_pol_LAT[j]+s4
print('LAT:', (ss_pol_LAT/ss1_solat)**(-1/2)-1)


#SAT
ss_pol_SAT=np.zeros(17)
for j in np.arange(len(ss_pol_SAT)):
    for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+unlensedTQU_SAT_alpha_TEB[j][2,i-1]))**2
        ss_pol_SAT[j]=ss_pol_SAT[j]+s4
print('SAT:', (ss_pol_SAT/ss1_sosat)**(-1/2)-1)

#PICO
ss_pol_PICO=np.zeros(17)
for j in np.arange(len(ss_pol_PICO)):
    for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+unlensedTQU_PICO_alpha_TEB[j][2,i-1]))**2
        ss_pol_PICO[j]=ss_pol_PICO[j]+s4
print('PICO:', (ss_pol_PICO/ss1_pico)**(-1/2)-1)



LiteBIRD: [8.5038221882660636e-05 7.6438977030846011e-04 2.1183052921152079e-03
 4.1374544126198121e-03 8.3830136492843987e-03 1.1998647209181312e-02
 1.6218216692945431e-02 2.1018845238912620e-02 2.6376631722678567e-02
 3.2267335382025530e-02 3.8666972778436781e-02 4.5552308114365614e-02
 5.2901229272642958e-02 6.0693011591234303e-02 6.8908478681678487e-02
 2.3428354234484017e-01 4.6325424324191355e-01]
DW: [7.0312834641095279e-05 6.3269772679253045e-04 1.7569113112638757e-03
 3.4418057907770194e-03 7.0165294738704365e-03 1.0094437375180343e-02
 1.3724660430558089e-02 1.7903635113818872e-02 2.2627316923268470e-02
 2.7891206947659564e-02 3.3690380587930946e-02 4.0019518011387367e-02
 4.6872935912671831e-02 5.4244620163582002e-02 6.2128258950207016e-02
 2.3698157651405927e-01 5.0109138019247035e-01]
UD: [3.2482927348098656e-04 2.9205797284541646e-03 8.0984544968891736e-03
 1.5831602922182553e-02 3.2133791252351607e-02 4.6060184040769059e-02
 6.2360026509994615e-02 8.0962636888012529e-02

# pointing

In [51]:

#1, 1.25, 1.5, 1.75,  2, 2.25, 2.5,  2.75, 3, 3.5, 4, 4.5,  5, 5.5, 6, 6.5, 7, 8, 9, 10, 15, 20 arcmin fwhm


#LiteBIRD
ss_pointing=np.zeros(len(TEB_pointing))
for j in np.arange(len(TEB_pointing)):
    for i in ls: 
        s8=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+TEB_pointing[j][2][i-1]))**2
        ss_pointing[j]=ss_pointing[j]+s8
print('LiteBIRD:', (ss_pointing/ss1_lite)**(-1/2)-1)

#SAT
ss_SAT_pointing=np.zeros(len(TEB_pointing))
for j in np.arange(len(TEB_pointing)):
    for i in ls: 
        s8=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+TEB_SAT_pointing[j][2][i-1]))**2
        ss_SAT_pointing[j]=ss_SAT_pointing[j]+s8
print('SAT:', (ss_SAT_pointing/ss1_sosat)**(-1/2)-1)

#PICO
ss_PICO_pointing=np.zeros(len(TEB_pointing))
for j in np.arange(len(TEB_pointing)):
    for i in ls: 
        s8=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+TEB_PICO_pointing[j][2][i-1]))**2
        ss_PICO_pointing[j]=ss_PICO_pointing[j]+s8
print('PICO:', (ss_PICO_pointing/ss1_pico)**(-1/2)-1)




LiteBIRD: [0.0013699603663737 0.0009742464875961 0.0010424204079289
 0.0008893095609432 0.0011444844188873 0.0011849003147268
 0.0014332624583164 0.0021674341664937 0.0034311034700438
 0.007702204460885  0.0140875962810103 0.0246467813956504
 0.0374074782108482 0.0503289531514433 0.0636585847837476
 0.0800129698626084 0.0946224538803617 0.1279712856819708
 0.1549165079554864 0.186265152999562  0.3391800825743709
 0.5140136681943037]
SAT: [2.7223474783548696e-03 2.5300915095531895e-03 2.4106926888609603e-03
 1.8620730544143260e-03 2.6468130727617112e-03 3.3869336198906197e-03
 6.7618231357697844e-03 1.2089847644126683e-02 2.1145036062111933e-02
 5.2573351779415356e-02 1.1242681133588861e-01 1.8661995745397775e-01
 2.8157371184194413e-01 3.8843058555663501e-01 4.7977845181170142e-01
 6.1378067373266498e-01 7.4322952115419527e-01 9.7044818408295508e-01
 1.1929691900183781e+00 1.3668668233382091e+00 2.4031242402047899e+00
 3.3251934577613733e+00]
PICO: [1.7228958524330151e-02 1.31661164522

# Beam

In [54]:
# beam 10 LiteBIRD
print('LiteBIRD')

ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_100GHz_10FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('100GHz 10deg',(ss2/ss1_lite)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_140GHz_10FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('140GHz 10deg',(ss2/ss1_lite)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_195GHz_10FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('195GHz 10deg',(ss2/ss1_lite)**(-1/2)-1)

# beam 15 LiteBIRD
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_100GHz_15FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('100GHz 15deg',(ss2/ss1_lite)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_140GHz_15FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('140GHz 15deg',(ss2/ss1_lite)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_195GHz_15FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('195GHz 15deg',(ss2/ss1_lite)**(-1/2)-1)

# beam 20 LiteBIRD
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_100GHz_20FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('100GHz 20deg',(ss2/ss1_lite)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_140GHz_20FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('140GHz 20deg',(ss2/ss1_lite)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+1/0.6*noises_map_195GHz_20FSL_mmasked_B_filtered[i-1]))**2
    ss2=ss2+s2
print('195GHz 20deg',(ss2/ss1_lite)**(-1/2)-1)



LiteBIRD
100GHz 10deg 0.03657899051375968
140GHz 10deg 0.05386193489131985
195GHz 10deg 0.1008490252045049
100GHz 15deg 0.10697168192095607
140GHz 15deg 0.13824242920242558
195GHz 15deg 0.21145944418298135
100GHz 20deg 0.19768718452836964
140GHz 20deg 0.24230615562412128
195GHz 20deg 0.35874268230291717


In [55]:
# beam 10 SO LAT
print('SO LAT')

ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_93GHz_10FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('93GHz 10deg',(ss2/ss1_solat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_145GHz_10FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 10deg',(ss2/ss1_solat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_225GHz_10FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('225GHz 10deg',(ss2/ss1_solat)**(-1/2)-1)

# beam 15 SO LAT
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_93GHz_15FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('93GHz 15deg',(ss2/ss1_solat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_145GHz_15FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 15deg',(ss2/ss1_solat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_225GHz_15FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('225GHz 15deg',(ss2/ss1_solat)**(-1/2)-1)

# beam 20 SO LAT
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_93GHz_20FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('93GHz 20deg',(ss2/ss1_solat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_145GHz_20FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 20deg',(ss2/ss1_solat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+1/0.4*noises_map_225GHz_20FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('225GHz 20deg',(ss2/ss1_solat)**(-1/2)-1)



SO LAT
93GHz 10deg 0.0001499337127701672
145GHz 10deg 0.00027198300764008643
225GHz 10deg 0.0010050525184923664
93GHz 15deg 0.001115035825400179
145GHz 15deg 0.0020224824042884837
225GHz 15deg 0.007468841196584863
93GHz 20deg 0.006739281462263991
145GHz 20deg 0.012216691310867889
225GHz 20deg 0.04495921079818421


In [56]:
# beam 10 SO SAT
print('SO SAT')

ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_93GHz_10FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('93GHz 10deg',(ss2/ss1_sosat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_145GHz_10FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 10deg',(ss2/ss1_sosat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_225GHz_10FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('225GHz 10deg',(ss2/ss1_sosat)**(-1/2)-1)

# beam 15 SO SAT
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_93GHz_15FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('93GHz 15deg',(ss2/ss1_sosat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_145GHz_15FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 15deg',(ss2/ss1_sosat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_225GHz_15FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('225GHz 15deg',(ss2/ss1_sosat)**(-1/2)-1)

# beam 20 SO SAT
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_93GHz_20FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('93GHz 20deg',(ss2/ss1_sosat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_145GHz_20FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 20deg',(ss2/ss1_sosat)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+1/0.1*noises_map_225GHz_20FSL_mmasked_TEB_SAT[2][i-1]))**2
    ss2=ss2+s2
print('225GHz 20deg',(ss2/ss1_sosat)**(-1/2)-1)



SO SAT
93GHz 10deg 0.056457126532804214
145GHz 10deg 0.0930303392371874
225GHz 10deg 0.2517701570163735
93GHz 15deg 0.1285978296216539
145GHz 15deg 0.20339534856547448
225GHz 15deg 0.537546381044236
93GHz 20deg 0.27618666550445137
145GHz 20deg 0.4285194307348126
225GHz 20deg 1.1206342921096426


In [59]:
# beam 10 S4 DW
print('S4 DW')

ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_95GHz_10FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('95GHz 10deg',(ss2/ss1_s4dw)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_145GHz_10FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 10deg',(ss2/ss1_s4dw)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_220GHz_10FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('220GHz 10deg',(ss2/ss1_s4dw)**(-1/2)-1)

# beam 15 S4 DW
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_95GHz_15FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('95GHz 15deg',(ss2/ss1_s4dw)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_145GHz_15FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 15deg',(ss2/ss1_s4dw)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_220GHz_15FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('220GHz 15deg',(ss2/ss1_s4dw)**(-1/2)-1)

# beam 20 S4 DW
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_95GHz_20FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('95GHz 20deg',(ss2/ss1_s4dw)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_145GHz_20FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 20deg',(ss2/ss1_s4dw)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+1/0.6*noises_map_220GHz_20FSL_mmasked_TEB_DW[2][i-1]))**2
    ss2=ss2+s2
print('220GHz 20deg',(ss2/ss1_s4dw)**(-1/2)-1)



S4 DW
95GHz 10deg 0.0003642471261187108
145GHz 10deg 0.0006485584812385703
220GHz 10deg 0.002182706702613224
95GHz 15deg 0.00272371514272729
145GHz 15deg 0.004848401947674441
220GHz 15deg 0.01629383264380002
95GHz 20deg 0.016533227169258202
145GHz 20deg 0.02938722632886903
220GHz 20deg 0.09801644603090542


In [57]:
# beam 10 S4 UD
print('S4 UD')

ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_95GHz_10FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('95GHz 10deg',(ss2/ss1_s4ud)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_145GHz_10FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 10deg',(ss2/ss1_s4ud)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_220GHz_10FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('220GHz 10deg',(ss2/ss1_s4ud)**(-1/2)-1)

# beam 15 S4 UD
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_95GHz_15FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('95GHz 15deg',(ss2/ss1_s4ud)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_145GHz_15FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 15deg',(ss2/ss1_s4ud)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_220GHz_15FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('220GHz 15deg',(ss2/ss1_s4ud)**(-1/2)-1)

# beam 20 S4 UD
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_95GHz_20FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('95GHz 20deg',(ss2/ss1_s4ud)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_145GHz_20FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('145GHz 20deg',(ss2/ss1_s4ud)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+1/0.03*noises_map_220GHz_20FSL_mmasked_TEB_UD[2][i-1]))**2
    ss2=ss2+s2
print('220GHz 20deg',(ss2/ss1_s4ud)**(-1/2)-1)



S4 UD
95GHz 10deg 0.020231461185967525
145GHz 10deg 0.03592605204010324
220GHz 10deg 0.1193415292740958
95GHz 15deg 0.149317904391582
145GHz 15deg 0.2619062288234757
220GHz 15deg 0.8367743666454557
95GHz 20deg 0.8449895690667912
145GHz 20deg 1.4419253404217098
220GHz 20deg 4.112438754439044


In [60]:
# beam 10 PICO
print('PICO')

ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_90GHz_10FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('90GHz 10deg',(ss2/ss1_pico)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_155GHz_10FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('155GHz 10deg',(ss2/ss1_pico)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_223GHz_10FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('223GHz 10deg',(ss2/ss1_pico)**(-1/2)-1)

# beam 15 PICO
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_90GHz_15FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('90GHz 15deg',(ss2/ss1_pico)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_155GHz_15FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('155GHz 15deg',(ss2/ss1_pico)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_223GHz_15FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('223GHz 15deg',(ss2/ss1_pico)**(-1/2)-1)

# beam 20 PICO
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_90GHz_20FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('90GHz 20deg',(ss2/ss1_pico)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_155GHz_20FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('155GHz 20deg',(ss2/ss1_pico)**(-1/2)-1)
ss2=0
for i in ls: 
    s2=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+1/0.6*noises_map_223GHz_20FSL_mmasked_TEB_PICO[2][i-1]))**2
    ss2=ss2+s2
print('223GHz 20deg',(ss2/ss1_pico)**(-1/2)-1)



PICO
90GHz 10deg 0.02406328687360415
155GHz 10deg 0.050441476894466764
223GHz 10deg 0.14509934271445712
90GHz 15deg 0.12545463390129608
155GHz 15deg 0.24008580298199855
223GHz 15deg 0.5732452292972487
90GHz 20deg 0.4275188380855546
155GHz 20deg 0.6956579784883106
223GHz 20deg 1.3480380069904974


# COMBINATION OF THE EFFECTS

In [70]:
# polarization angle 
# [1,3,5,7,10,12,14,16,18,20,22,24,26,28,30,60,90] arcmin

# pointing
#[1, 1.25, 1.5, 1.75,  2, 2.25, 2.5,  2.75, 3, 3.5, 4, 4.5,  5, 5.5, 6, 6.5, 7, 8, 9, 10, 15, 20] arcmin fwhm


#LiteBIRD real
ss_tot_lite=np.zeros(1)
ss_tot_lite1=np.zeros(1)
ss_tot_DW=np.zeros(1)
ss_tot_UD=np.zeros(1)
ss_tot_LAT=np.zeros(1)
ss_tot_SAT=np.zeros(1)
ss_tot_PICO=np.zeros(1)


for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+unlensedTQU_alpha_TEB[4,2,i-1]+TEB_pointing[9][2][i-1]+1/0.6*noises_map_140GHz_20FSL_mmasked_B_filtered[i-1]))**2
        ss_tot_lite=ss_tot_lite+s4
print('LiteBIRD real :', (ss_tot_lite/ss1_lite)**(-1/2)-1)

#LiteBIRD
for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[0]*clbb[i-1]/(ant(i-1)[0]*RF[0]*foreCL[i-1]+ant(i-1)[0]*Dauto[0]*lenCL[i-1]+noiseCL+unlensedTQU_alpha_TEB[4,2,i-1]+TEB_pointing[9][2][i-1]+1/0.6*noises_map_140GHz_10FSL_mmasked_B_filtered[i-1]))**2
        ss_tot_lite1=ss_tot_lite1+s4
print('LiteBIRD:', (ss_tot_lite1/ss1_lite)**(-1/2)-1)

#DW
for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[1]*clbb[i-1]/(ant(i-1)[1]*RF[1]*foreCL[i-1]+ant(i-1)[1]*Dauto[1]*lenCL[i-1]+(2.8/2)**2*noiseCL+unlensedTQU_DW_alpha_TEB[5,2,i-1]+1/0.6*noises_map_145GHz_15FSL_mmasked_TEB_DW[2][i-1]))**2
        ss_tot_DW=ss_tot_DW+s4
print('DW:', (ss_tot_DW/ss1_s4dw)**(-1/2)-1)

#UD
for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[2]*clbb[i-1]/(ant(i-1)[2]*RF[2]*foreCL[i-1]+ant(i-1)[2]*Dauto[2]*lenCL[i-1]+(0.96/2)**2*noiseCL+unlensedTQU_UD_alpha_TEB[2,2,i-1]+1/0.03*noises_map_145GHz_10FSL_mmasked_TEB_UD[2][i-1]))**2
        ss_tot_UD=ss_tot_UD+s4
print('UD:', (ss_tot_UD/ss1_s4ud)**(-1/2)-1)


#LAT (POLAR NOISE SAME AS DW BUT SENSITIVITY IS DIFFERENT)
for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[3]*clbb[i-1]/(ant(i-1)[3]*RF[3]*foreCL[i-1]+ant(i-1)[3]*Dauto[3]*lenCL[i-1]+(6/2)**2*noiseCL+unlensedTQU_DW_alpha_TEB[10,2,i-1]+1/0.4*noises_map_145GHz_20FSL_mmasked_TEB_DW[2][i-1]))**2
        ss_tot_LAT=ss_tot_LAT+s4
print('LAT:', (ss_tot_LAT/ss1_solat)**(-1/2)-1)


#SAT
for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[4]*clbb[i-1]/(ant(i-1)[4]*RF[4]*foreCL[i-1]+ant(i-1)[4]*Dauto[4]*lenCL[i-1]+(2/2)**2*noiseCL+unlensedTQU_SAT_alpha_TEB[5,2,i-1]+TEB_SAT_pointing[7][2][i-1]+1/0.1*noises_map_145GHz_10FSL_mmasked_TEB_SAT[2][i-1]))**2
        ss_tot_SAT=ss_tot_SAT+s4
print('SAT:', (ss_tot_SAT/ss1_sosat)**(-1/2)-1)

#PICO
for i in ls: 
        s4=(2*i+1)/2*(ant(i-1)[5]*clbb[i-1]/(ant(i-1)[5]*RF[5]*foreCL[i-1]+ant(i-1)[5]*Dauto[5]*lenCL[i-1]+(0.61/2)**2*noiseCL+unlensedTQU_PICO_alpha_TEB[1,2,i-1]+TEB_PICO_pointing[1][2][i-1]+1/0.6*noises_map_155GHz_10FSL_mmasked_TEB_PICO[2][i-1]))**2
        ss_tot_PICO=ss_tot_PICO+s4
print('PICO:', (ss_tot_PICO/ss1_pico)**(-1/2)-1)



LiteBIRD real : [0.2591172683161569]
LiteBIRD: [0.0694597765606972]
DW: [0.0149262626857147]
UD: [0.0439942122163175]
LAT: [0.021779065633222]
SAT: [0.1163774186900925]
PICO: [0.0707538003551249]
